# Reading and Writing Audio Files with PySoundFile

[back to overview page](index.ipynb)

http://github.com/bastibe/PySoundFile

TODO: downloading and installing PySoundFile

Advantages: 

* supports many file formats (thanks to libsndfile)
* supports 24-bit PCM and 32-bit floating point WAV files
* WAVEX support
* can read parts of audio files
* automatic type conversion and normalization

Disadvantages:

* has to be downloaded and installed manually (but it's easy)
* not widely known (yet!)
* needs NumPy (which isn't really a problem at all)

## Reading

This is the quickest way to load a WAV file into a NumPy array:

In [ ]:
from pysoundfile import SoundFile
sig = SoundFile('data/test_wav_pcm16.wav')[:]

That's all. Easy, isn't it?

But let's have a closer look ...

In [ ]:
%pylab inline

In [ ]:
plot(sig)
show()

Looking good!

In [ ]:
w = SoundFile('data/test_wav_pcm16.wav')

In [ ]:
w.channels, w.frames, w.sample_rate

In [ ]:
w.format

In [ ]:
sig.shape

In [ ]:
test = w.read()

In [ ]:
test.shape

In [ ]:
plot(test)
show()

OK, read() works as expected.

In [ ]:
# TODO: read mono file

Maybe, it might be better if a mono sound file would turn into a one-dimensional NumPy array?
This would be consistent with the rest of NumPy (although not expected by Octave users).

In [ ]:
# TODO: load 24-bit file 'data/test_wav_pcm24.wav'

In [ ]:
# TODO: load WAVEX 'data/test_wavex_pcm16.wav' and 'data/test_wavex_pcm24.wav'

In [ ]:
# TODO: load 32-bit float files 'test_wav_float32.wav' and 'test_wavex_float32.wav'

In [ ]:
# TODO: test PyPy support

## Writing

In [ ]:
# TODO!

## Version Info

In [ ]:
import pysoundfile

import sys, IPython
print("Versions: NumPy = {}; IPython = {}".format(numpy.__version__, IPython.__version__))

print("Python interpreter:")
print(sys.version)

try:
    print("PySoundFile version =", pysoundfile.__version__)
except:
    import traceback
    traceback.print_exc()

Bug report for missing version number: https://github.com/bastibe/PySoundFile/issues/7

In [ ]:
pysoundfile.snd_types

In [ ]:
pysoundfile.snd_subtypes

<p xmlns:dct="http://purl.org/dc/terms/">
  <a rel="license"
     href="http://creativecommons.org/publicdomain/zero/1.0/">
    <img src="http://i.creativecommons.org/p/zero/1.0/88x31.png" style="border-style: none;" alt="CC0" />
  </a>
  <br />
  To the extent possible under law,
  <span resource="[_:publisher]" rel="dct:publisher">
    <span property="dct:title">Matthias Geier</span></span>
  has waived all copyright and related or neighboring rights to
  this work.
</p>